#Hate Speech Detection with AraBERT and HuggingFace
In this assignment, we will be exploring the application of the AraBERT model specifically for the task of hate speech detection. We will use the AJGT Sentiment Analysis dataset from K. M. Alomari, H. M. ElSherif, and K. Shaalan, “Arabic tweets sentimental analysis using machine learning,” in Proceedings of the International Conference on Industrial, Engineering and Other Applications of Applied Intelligent Systems, pp. 602–610, Montreal, Canada, June 2017.


# Check which GPU we have

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Wed Jan 24 22:45:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-------

##Installing Prerequisites

In [2]:
!pip install transformers[torch]
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

fatal: destination path 'arabert' already exists and is not an empty directory.


#Reading Data
We will rely on the following libraries for training and evaluation:

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
!git clone https://github.com/komari6/Arabic-twitter-corpus-AJGT.git

fatal: destination path 'Arabic-twitter-corpus-AJGT' already exists and is not an empty directory.


In [5]:
!ls Arabic-twitter-corpus-AJGT

AJGT.xlsx  LICENSE.md  README.md


**Question 1:** Read the dataset and arrange the columns name using the set variables:

In [6]:
dataset= pd.read_excel('Arabic-twitter-corpus-AJGT/AJGT.xlsx')

DATA_COLUMN = 'text'
LABEL_COLUMN = 'labels'
dataset=dataset[['Feed' , 'Sentiment']]
dataset.columns = [DATA_COLUMN, LABEL_COLUMN]
dataset.head()

,text,labels
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,كله رائع بجد ربنا يكرمك,Positive
3,لسانك قذر يا قمامه,Negative
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative


In [7]:
dataset.shape

(1800, 2)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1800 non-null   object
 1   labels  1800 non-null   object
dtypes: object(2)
memory usage: 28.2+ KB


**Question 2**: Split the data into training and testing (80-20)

In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42)

**Question 3:** Plot the distribution of lengths of sentences in both training and test set. Extract the max_len value to be used later:

maximum training length: 864
maximum testing length: 683


In [10]:
# plotting histogram for the sentences frequencies
plt.hist(train_df[DATA_COLUMN].str.len(), bins=10)
plt.hist(test_df[DATA_COLUMN].str.len(), bins=10)
plt.title('sentences distributions')
plt.xlabel('length')
plt.ylabel('frequency')
plt.legend(['train', 'test'])
plt.show()

max_train=train_df[DATA_COLUMN].str.len().max()
max_test=test_df[DATA_COLUMN].str.len().max()
print('maximum training length:',max_train)
print('maximum testing length:',max_test)

 We can conclude that the max_len can be chosen to be **256 ** since bert model has 512 tokens it would not eccept text longer than this number .

#Training Requirements

Let's start by creating the dataset needed for training and testing, we will use the Dataset class from pytorch as our base class.

For tokenization, we will be using the autotokenizer from HuggingFace.

In [11]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers.data.processors.utils import InputFeatures
from transformers import AutoTokenizer

#define the Dataset class
class SADataset(Dataset):
  def __init__(self, texts, labels, model_name, max_len, label_map):
    #hold the text and reviews inside the dataset class
    self.texts = texts
    self.labels = labels
    self.label_map = label_map
    self.tokenizer_name = model_name
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.max_len = max_len

  def __len__(self):
    #returns the length of the dataset
    return len(self.texts)

  def __getitem__(self, item):
    # Select the example based on the item ID
    text = str(self.texts[item])
    label = self.labels[item]

    input_dict = self.tokenizer(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation= True
      )

    return InputFeatures(input_ids=input_dict["input_ids"],
                         token_type_ids=input_dict['token_type_ids'],
                         attention_mask=input_dict["attention_mask"],
                         label=self.label_map[self.labels[item]])




**Question 4:** Define the evaluation metrics that we will need, including `accuracy_score`, `f1_score`, `precision_score` and `recall_score` from sklearn.

In [12]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    assert len(preds) == len(pred.label_ids)
 # label check with metrices define
    macro_f1 = f1_score(pred.label_ids, preds, average='macro')
    macro_f1_pos_neg = f1_score(pred.label_ids, preds, average='binary',labels=[0,1])
    macro_precision = precision_score(pred.label_ids, preds, average='macro')
    macro_recall = recall_score(pred.label_ids, preds, average='macro')
    acc = accuracy_score(pred.label_ids, preds)

    return {
        'macro_f1': macro_f1,
        'macro_f1_pos_neg': macro_f1_pos_neg,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'accuracy': acc
    }



## Preprocess the dataset
Let's start by defining the AraBERT preprocessor:

In [13]:
from arabert.preprocess import ArabertPreprocessor

model_name = 'aubmindlab/bert-base-arabertv02'
arabert_prep = ArabertPreprocessor(model_name)

**Question 5:** Apply preprocessing for the text column using the arabert preprocessor:

In [14]:
train_df[DATA_COLUMN] = train_df[DATA_COLUMN].apply(lambda x: arabert_prep.preprocess(x) )
test_df[DATA_COLUMN] = test_df[DATA_COLUMN].apply(lambda x: arabert_prep.preprocess(x))

In [15]:
train_df[DATA_COLUMN][:3] # as we see these 3 sentences after preprpcessing using Arabert preprocesser

832     سبحان الله بحمده عدد خلقه رضى نفسه زنه عرشه مد...
836                         سبحان الله مالك السموات الارض
1103    قصه جميله جدا تعكس معنى الايمان التمسك بالعقيد...
Name: text, dtype: object

Let's extract the label to id map:

In [16]:
label_list = list(test_df['labels'].unique())
label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

{'Negative': 0, 'Positive': 1}


**Question 6:** Create the train and test SADataset:

In [17]:
max_len = 256
train_dataset = SADataset(texts=train_df['text'].tolist(),
                          labels=train_df['labels'].tolist(),
                          model_name=model_name,
                          max_len=max_len,
                          label_map=label_map)




test_dataset = SADataset(texts=test_df['text'].tolist(),
                         labels=test_df['labels'].tolist(),
                         model_name=model_name,
                         max_len=max_len,
                         label_map=label_map)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Setup the HuggingFace trainer

We will be using the `bert-base-arabertv02` from HuggingFace models by Antoun et Al (2020). We can choose other Arabic BERT models by just changing the path here from `https://huggingface.co/models`.

In [18]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We can now set up the training arguments, you can more information from https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments

In [19]:
from transformers import Trainer , TrainingArguments

training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 5e-5,
    fp16 = True,
    per_device_train_batch_size = 20, # i try with 16 and 20 to see the difference
    per_device_eval_batch_size = 20,
    gradient_accumulation_steps = 2,
    num_train_epochs= 3, # change the epochs number to 3
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_macro_f1',
    greater_is_better = True,
    seed = 42
  )

In [20]:
training_args.__dict__

{'output_dir': './train',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': True,
 'do_predict': False,
 'evaluation_strategy': <IntervalStrategy.EPOCH: 'epoch'>,
 'prediction_loss_only': False,
 'per_device_train_batch_size': 20,
 'per_device_eval_batch_size': 20,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 2,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'learning_rate': 5e-05,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 3,
 'max_steps': -1,
 'lr_scheduler_type': <SchedulerType.LINEAR: 'linear'>,
 'warmup_ratio': 0.0,
 'warmup_steps': 0,
 'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': './train/runs/Jan24_22-46-29_7c3819129a86',
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': False,
 'logging_steps': 500,
 'logging_nan_inf_filter': True,
 

**Question:** Initialize the Trainer and start training:

In [21]:
from transformers import Trainer, TrainingArguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
 # train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy
1,No log,0.167836,0.930271,0.934726,0.929876,0.930796,0.930556
2,No log,0.139659,0.947006,0.950392,0.946594,0.947550,0.947222
3,No log,0.149207,0.952378,0.956743,0.954375,0.951118,0.952778


TrainOutput(global_step=108, training_loss=0.1775768068101671, metrics={'train_runtime': 132.5337, 'train_samples_per_second': 32.595, 'train_steps_per_second': 0.815, 'total_flos': 568319879577600.0, 'train_loss': 0.1775768068101671, 'epoch': 3.0})

#  Saving the best model

Before saving the model, let's change the label2id in the config file, and get the id to label map.

In [22]:
trainer.model.config.label2id = label_map
inv_label_map = { v:k for k, v in label_map.items()}
trainer.model.config.id2label = inv_label_map

In [23]:
#save the model in the folder
trainer.save_model("best_sa_model")
test_dataset.tokenizer.save_pretrained("best_sa_model")

('best_sa_model/tokenizer_config.json',
 'best_sa_model/special_tokens_map.json',
 'best_sa_model/vocab.txt',
 'best_sa_model/added_tokens.json',
 'best_sa_model/tokenizer.json')

# Loading the model for inference
We can use HuggingFace pipelines to load the model for inference:

In [24]:
from transformers import pipeline
pipe = pipeline(
        "sentiment-analysis",
        model = "best_sa_model",
        device=0, # set device to 0 for CUDA
        )

In [25]:
pipe("انا لا احبك")

[{'label': 'Negative', 'score': 0.9820467233657837}]

In [28]:
pipe("   الأحلام سوف تتحقق")

[{'label': 'Positive', 'score': 0.9953195452690125}]